## Home work 2: Word Counting




The map/reduce model is particularly well suited to applications like counting words in a document. 

All operations in Spark operate on data structures called RDDs, *Resilient Distributed
Datasets*. An RDD is nothing more than a collection of objects.  If an RDD contains only two-element tuples, the RDD is known
as a “pair RDD” and offers some additional functionality. The first element of each tuple
is treated as a key, and the second element as a value. 

The first step of every such Spark application is to create a Spark context:

In [1]:
import re
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf()
sc = SparkContext(conf=conf)

23/10/10 14:56:02 WARN Utils: Your hostname, yongyuangendangzouxinzhongyoudangshiyelixiang.local resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
23/10/10 14:56:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/10 14:56:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/10 14:56:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Next, you’ll need to read the target file into an RDD:

In [3]:
data = sc.textFile("./pg100.txt")

In [4]:
data.take(10)

['Project Gutenberg’s The Complete Works of William Shakespeare, by William Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever.  You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this eBook or online at',
 'www.gutenberg.org.  If you are not located in the United States, you’ll',
 'have to check the laws of the country where you are located before using',
 'this ebook.',
 '']

You now have an RDD filled with strings, one per line of the file.
Next you’ll want to split the lines into individual words:

In [5]:
words = data.flatMap(lambda line: re.split(r'[^\w]+', line))

In [6]:
words.count()

1185725

In [7]:
caps = words.map(lambda word: word.upper())
final = caps.filter(lambda x: x!='')

The flatMap() operation first converts each line into an array of words, and then makes
each of the words an element in the new RDD. If you asked Spark to count the number of
elements in the words RDD, it would tell you the number of words in the file.
Next, you’ll want to replace each word with a tuple of that word and the number 1.

In [8]:
pairs = final.map(lambda x: (x,1))

In [9]:
pairs.take(10)

[('PROJECT', 1),
 ('GUTENBERG', 1),
 ('S', 1),
 ('THE', 1),
 ('COMPLETE', 1),
 ('WORKS', 1),
 ('OF', 1),
 ('WILLIAM', 1),
 ('SHAKESPEARE', 1),
 ('BY', 1)]

The map() operation replaces each word with a tuple of that word and the number 1. The
pairs RDD is a pair RDD where the word is the key, and all of the values are the number
1.

Now, to get a count of the number of instances of each word, you need only group the
elements of the RDD by key (word) and add up their values:

In [10]:
counts = pairs.reduceByKey(lambda x, y: x + y)

In [11]:
counts.count()

26728

In [12]:
#counts.distinct().count()

The reduceByKey() operation keeps adding elements’ values together until there are no
more to add for each key (word).
Finally, you can store the results in a file and stop the context:

In [13]:
sc.stop()

## Task: 

Using Spark write a code that outputs the number of words that start with each letter. This means that for every letter we want to count the total number of (non-unique) words that start with that letter. In your implementation ignore the letter case,i.e., consider all words as lower case. You can ignore all non-alphabetic characters.